<a href="https://colab.research.google.com/github/rushikeshnaik779/new_water/blob/main/mistral_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚡ Deploy Mistral (Mixtral-8x7b) in a Notebook
Below is working code for running mixtral-8x7b locally (in a python notebook) with minimal requirements.


# 💻 Install requirements
Use huggingface-hub and llama-cpp-python to download the model and interface with it using llama-cpp (for inference)

In [ ]:
!pip3 install huggingface-hub
!pip install llama-cpp-python

In [ ]:
!huggingface-cli download TheBloke/dolphin-2.5-mixtral-8x7b-GGUF dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


# 💬 Chat with LLM
Set up a simple chat interaction using the mixtral artifact wrapped with llama-cpp python bindings. This may take a few minutes to load as the model since the artifact is downloaded.

It is recommended to use GPU accelerated runtime (i.e. V100)

Credit for below inference code goes to: https://huggingface.co/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/blob/main/README.md


In [ ]:
from llama_cpp import Llama
import os

# Set the path to your downloaded model file
model_path = "dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf"

# Initialize the Llama model
# Adjust these parameters based on your system capabilities and the model's requirements
# Below should work for V100 runtime
llm = Llama(
    model_path=model_path,
    n_ctx=32768,  # Max sequence length
    n_threads=2,  # Number of CPU threads
    n_gpu_layers=10  # Number of layers to offload to GPU (set to 0 if no GPU)
)

prompt = "system\n{system_message}\nuser\n{prompt}\nassistant"

# Function to append messages to the prompt and get the model's response
def chat_with_model(llm, prompt, user_message):
    # Append the user's message to the prompt
    updated_prompt = prompt + "\nuser\n" + user_message + "\nassistant"

    # Generate the model's response
    output = llm(
        updated_prompt,
        max_tokens=512,  # Adjust as needed
        stop=["</s>"],   # Stop token, adjust as needed
        echo=True        # Echo the prompt in the output
    )

    # Extract only the model's response from the dictionary
    full_response = output['choices'][0]['text'].strip()

    # Extract the latest response (after the last "assistant" tag)
    latest_response = full_response.split("assistant\n")[-1].strip()

    # Append the user's message and the model's latest response to the prompt for the next iteration
    updated_prompt += "\nuser\n" + user_message + "\nassistant\n" + latest_response

    return updated_prompt, latest_response

# Start the chat interaction
while True:
    # Get user input
    user_message = input("You: ")

    # Check for a special command to end the chat (e.g., 'quit')
    if user_message.lower() == 'quit':
        break

    # Get model response and update the prompt
    prompt, model_response = chat_with_model(llm, prompt, user_message)
    print("Model:", model_response)

In [ ]:
# double check the correct llm is loaded in
print(llm)
print("Model being used:", model_path)
